# Despacho múltiple, tipos abstractos y duck typing

[https://docs.julialang.org/en/v1/manual/methods/](https://docs.julialang.org/en/v1/manual/methods/)

[https://www.oxinabox.net/2020/02/09/whycompositionaljulia.html#multiple-dispatch--duck-typing](https://www.oxinabox.net/2020/02/09/whycompositionaljulia.html#multiple-dispatch--duck-typing)

## Despacho múltiple

En Julia, las siguientes definiciones son simultaneamente posibles.

In [1]:
f(x::Int,y::Int) = println("x e y son enteros")
f(x::Real,y::Int) = println("x es real e y es entero")
f(x::Int,y::Real) = println("x es entero e y es real")
f(x::Real,y::Real) = println("x e y son reales")

f (generic function with 4 methods)

Cada una de ella define un *método* de la función *genérica* `f`.

Durante un llamado a `f`, Julia selecciona el método a evaluar de acuerdo a los tipos de los argumentos especificados.

In [2]:
f(3,4)

x e y son enteros


In [3]:
f(π,4)

x es real e y es entero


In [4]:
f(3,√2)

x es entero e y es real


In [5]:
f(3.3,π)

x e y son reales


Esto se llama **despacho múltiple** (multiple dispatch). Si queremos ver los métodos de una función, escribimos:

In [6]:
methods(f)

# 4 methods for generic function "f":
[1] f(x::Int64, y::Int64) in Main at In[1]:1
[2] f(x::Real, y::Int64) in Main at In[1]:2
[3] f(x::Int64, y::Real) in Main at In[1]:3
[4] f(x::Real, y::Real) in Main at In[1]:4

In [7]:
methods(cos)

# 12 methods for generic function "cos":
[1] cos(x::BigFloat) in Base.MPFR at mpfr.jl:744
[2] cos(::Missing) in Base.Math at math.jl:1167
[3] cos(a::Complex{Float16}) in Base.Math at math.jl:1115
[4] cos(a::Float16) in Base.Math at math.jl:1114
[5] cos(z::Complex{T}) where T in Base at complex.jl:823
[6] cos(x::T) where T<:Union{Float32, Float64} in Base.Math at special/trig.jl:100
[7] cos(x::Real) in Base.Math at special/trig.jl:124
[8] cos(A::LinearAlgebra.Hermitian{#s663,S} where S<:(AbstractArray{#s664,2} where #s664<:#s663) where #s663<:Complex) in LinearAlgebra at /usr/share/julia/stdlib/v1.4/LinearAlgebra/src/symmetric.jl:907
[9] cos(A::Union{LinearAlgebra.Hermitian{#s664,S}, LinearAlgebra.Symmetric{#s664,S}} where S where #s664<:Real) in LinearAlgebra at /usr/share/julia/stdlib/v1.4/LinearAlgebra/src/symmetric.jl:903
[10] cos(D::LinearAlgebra.Diagonal) in LinearAlgebra at /usr/share/julia/stdlib/v1.4/LinearAlgebra/src/diagonal.jl:561
[11] cos(A::AbstractArray{#s664,2} where #s664<:Real) in LinearAlgebra at /usr/share/julia/stdlib/v1.4/LinearAlgebra/src/dense.jl:793
[12] cos(A::AbstractArray{#s664,2} where #s664<:Complex) in LinearAlgebra at /usr/share/julia/stdlib/v1.4/LinearAlgebra/src/dense.jl:800

### La expresividad del despacho multiple

| dispatch degree | syntax | dispatch arguments | expressive order | expressive power | lenguaje |
|--|--|--|--|--|--|
| none | $f(x_1$,$x_2$,$\dots)$ | {} | $\mathcal{O}(1)$ | constant | C, FORTRAN (< 2003) |
| single | $x_1$.$f(x_2$,$\dots)$ | {$x_1$} | $\mathcal{O}(|X_1|)$ | linear | C++, FORTRAN (2003), Java |
| multiple | $f(x_1$,$x_2$,$\dots)$ | {$x_1$,$x_2$,...}    | $\mathcal{O}(|X_1||X_2|\dots)$ | exponential | Julia, C# (4.0), LISP |

### Inferencia de tipos, compilación AOT y JIT

Para evaluar una expresión, Julia primero busca determinar la composición de métodos a evaluar en tiempo de compilación. En este sentido, Julia aplica *Ahead of Time (AOT) compilation*.

Cuando el tipo de algunos de los objetos involucrados se definen en tiempos de ejecución, AOT no puede aplicarse. Entonces, Julia utiliza un algoritmo de *infierencia de tipos* (type inference) para determinar la composición de métodos en tiempo de ejecución utilizando *Just In Time (JIT) compilation*.

## Tipos concretos y abstractos

El despacho múltiple es una herramienta poderosa si se lo combina con la noción de *tipos abstractos* (abstract types), los cuales sirven para definir jerarquías de tipos, subtipos, subsubtipos, etc.

Los *tipos concretos* no poseen subtipos y varios de ellos vienen predefinidos (`Int64`, `Float64`,`String`, etc.). 

Para tener una idea de como funciona múltiple dispatch, consideremos el siguiente ejemplo:

* [https://www.youtube.com/watch?v=kc9HwsxE1OY](https://www.youtube.com/watch?v=kc9HwsxE1OY)

In [8]:
abstract type Pet end

struct Dog <: Pet
    name::String
end

struct Cat <: Pet
    name::String
end

function encounter(a::Pet,b::Pet)
    verb = meets(a,b) 
    println("$(a.name) meets $(b.name) and $verb.")
end

meets(a::Dog,b::Dog) = "sniff"
meets(a::Dog,b::Cat) = "chases"
meets(a::Cat,b::Dog) = "hisses"
meets(a::Cat,b::Cat) = "slinks"

meets (generic function with 4 methods)

In [9]:
fido = Dog("Fido")
rex = Dog("Rex")
whiskers = Cat("Whiskers")
spots = Cat("Spots")

encounter(fido,rex)
encounter(fido,whiskers)
encounter(rex,spots)
encounter(whiskers,spots)

Fido meets Rex and sniff.
Fido meets Whiskers and chases.
Rex meets Spots and chases.
Whiskers meets Spots and slinks.


## Duck typing

Consideremos el siguiente ejemplo:

* [https://www.oxinabox.net/2020/02/09/whycompositionaljulia.html#multiple-dispatch--duck-typing](https://www.oxinabox.net/2020/02/09/whycompositionaljulia.html#multiple-dispatch--duck-typing)

* [https://www.compart.com/en/unicode/U+1F986](https://www.compart.com/en/unicode/U+1F986)

Supongamos que alguna libreria escrita por terceros implementa el tipo `Duck` (pato) y algunas de sus funcionalidades.

In [10]:
struct Duck end

draw(duck) = "🦆"
function talk(duck)  
    println(draw(duck)," quack")
end
function raise_young(duck,babe) 
    println(draw(duck)," leads 🐤") 
end

raise_young (generic function with 1 method)

In [11]:
talk(Duck())

🦆 quack


In [12]:
raise_young(Duck(),Duck())

🦆 leads 🐤


Nos interesa extender la librería (sin tocarla) agregando el tipo `Swan` (cisne).

In [13]:
struct Swan end

In [14]:
talk(Swan())

🦆 quack


Esto no está bien!

Para corregir esto hacemos un poco de *duck typing*.

In [15]:
draw(swan::Swan) = "🦢"
function talk(swan::Swan)
    println(show(swan)," hiss")
end

talk (generic function with 2 methods)

In [16]:
talk(Swan())

Swan()nothing hiss


Ahora esta mejor.

Probemos ver que pasa cuando hay cisnes bebes

In [17]:
raise_young(Swan(),Swan())

🦢 leads 🐤


Vemos que surge otro problema. Los cisnes no caminan delante de sus pequeños, los llevan a cococho!

Para corregirlo, escribimos

In [18]:
function raise_young(swan::Swan,babe) 
    println(draw(swan)," carry 🐤")
end

raise_young (generic function with 2 methods)

y volvemos a probar

In [19]:
raise_young(Swan(),Swan())

🦢 carry 🐤


Que pasa si mezclamos especies?

In [20]:
raise_young(Duck(),Swan())

🦆 leads 🐤


De nuevo vemos un problema. Sabemos que los patos abandonan a los cisnes bebés.

Corregimos el problema, escribiendo

In [21]:
function raise_young(swan::Duck,babe::Swan)
    println(draw(swan)," abandon 🐤")
end

raise_young (generic function with 3 methods)

y volvemos a probar

In [22]:
raise_young(Duck(),Swan())

🦆 abandon 🐤


Que pasa si invertimos el orden y pedimos que un cisne críe un patito?

In [23]:
raise_young(Swan(),Duck())

🦢 carry 🐤


## El problema de la extensibilidad

*The Expression Problem Revisited, M. Torgersen (2004)*

En que grado nuestro paquete de código puede ser extructurado de manera tal que:

1. el **modelo de datos** y

2. el conjunto de **operaciones (virtuales)** sobre ellos

puedan ser **extendidos** sin necesidad de:

1. modificar el código ya existente,

2. repetir código y

3. sin introducir errores de tipo en tiempos de ejecución?

#### En otras palabras

Se puede fácil y "correctamente" hacer las siguientes cosas:

1. definir **tipos nuevos** sobre los cuales las **operaciones existentes** apliquen, y

2. definir **nuevas operaciones** que apliquen sobre los **tipos existentes**?

Este tipo de requermientos resultan fáciles/díficiles de implementar según el paradigma utilizado:

1. es **fácil** en lenguajes **orientados a objetos** y **difícil** en **lenguajes funcionales**, y

2. es **difícil** en lenguajes **orientados a objetos** y **fácil** en **lenguajes funcionales**.

#### Despacho múltiple propone lo mejor de ambos mundos:

1. Agregar nuevos métodos a las **funciones/operaciones genéricas existentes** para actuar sobre **tipos nuevos**.

2. Agregar nuevos métodos a **nuevas funciones/operaciones genéricas** que actúen sobre **tipos existentes**.

### Ejemplo real: DifferentialEquations + Measurements

Los paquetes `DifferentialEquations` y `Measurements` fueron implementados de manera independiente. El primero se desarrolló para resolver ecuaciones diferenciales, el segundo para lidiar con datos que tengan incerteza experimental (ej. $x = 1.35 \pm 0.1$). 

Como el paquete `Measurements` es un subtipo de `Number`, incorporando métodos nuevos para modelar operaciones matemáticas con ellos (ej. propagación de errores), el integrador ODE proveido en `DifferentialEquations` puede aplicarse directamente a mediciones:

In [24]:
using DifferentialEquations
using Measurements

ArgumentError: [91mArgumentError: Package DifferentialEquations not found in current path:[39m
[91m- Run `import Pkg; Pkg.add("DifferentialEquations")` to install the DifferentialEquations package.[39m


In [25]:
g = 9.79 ± 0.02
L = 1.0 ± 0.01

u₀ = [0 ± 0, π/60 ± 0.01]
tspan = (0.0,6.3)

function pendulum(du,u,p,t)
    θ = u[1] 
    dθ = u[2]
    du[1] = dθ
    du[2] = -(g/L)*θ
end

prop = ODEProblem(pendulum, u₀, tspan)
sol = solve(prop, Tsit5(), reltol = 1e-6)

UndefVarError: [91mUndefVarError: ± not defined[39m

In [26]:
using Plots

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1260


In [27]:
sol_u2 = reduce(hcat,sol.u)'[:,2]

UndefVarError: [91mUndefVarError: sol not defined[39m

In [28]:
using LaTeXStrings
scatter(sol.t,sol_u2,label="",xlabel=L"t",ylabel=L"u_2")

ArgumentError: [91mArgumentError: Package LaTeXStrings not found in current path:[39m
[91m- Run `import Pkg; Pkg.add("LaTeXStrings")` to install the LaTeXStrings package.[39m


In [29]:
methods(+)

# 222 methods for generic function "+":
[1] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:286
[2] +(x::Bool, y::Bool) in Base at bool.jl:96
[3] +(x::Bool) in Base at bool.jl:93
[4] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:104
[5] +(x::Bool, z::Complex) in Base at complex.jl:293
[6] +(a::Float16, b::Float16) in Base at float.jl:398
[7] +(x::Float32, y::Float32) in Base at float.jl:400
[8] +(x::Float64, y::Float64) in Base at float.jl:401
[9] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:287
[10] +(z::Complex{Bool}, x::Real) in Base at complex.jl:301
[11] +(::Missing, ::Missing) in Base at missing.jl:114
[12] +(::Missing) in Base at missing.jl:100
[13] +(::Missing, ::Number) in Base at missing.jl:115
[14] +(level::Base.CoreLogging.LogLevel, inc::Integer) in Base.CoreLogging at logging.jl:107
[15] +(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:414
[16] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:515
[17] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:514
[18] +(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:513
[19] +(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:473
[20] +(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:521
[21] +(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:527
[22] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:564
[23] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:557
[24] +(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:551
[25] +(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:410
[26] +(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:379
[27] +(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:386
[28] +(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:394
[29] +(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:402
[30] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod) in Dates at /usr/share/julia/stdlib/v1.4/Dates/src/periods.jl:341
[31] +(x::Dates.CompoundPeriod, y::Dates.Period) in Dates at /usr/share/julia/stdlib/v1.4/Dates/src/periods.jl:339
[32] +(x::Dates.CompoundPeriod, y::Dates.TimeType) in Dates at /usr/share/julia/stdlib/v1.4/Dates/src/periods.jl:369
[33] +(x::Dates.Date, y::Dates.Day) in Dates at /usr/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:74
[34] +(x::Dates.Date, y::Dates.Week) in Dates at /usr/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:72
[35] +(dt::Dates.Date, z::Dates.Month) in Dates at /usr/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:55
[36] +(dt::Dates.Date, y::Dates.Year) in Dates at /usr/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:28
[37] +(dt::Dates.Date, t::Dates.Time) in Dates at /usr/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:19
[38] +(t::Dates.Time, dt::Dates.Date) in Dates at /usr/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:20
[39] +(x::Dates.Time, y::Dates.TimePeriod) in Dates at /usr/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:78
[40] +(dt::Dates.DateTime, z::Dates.Month) in Dates at /usr/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:48
[41] +(dt::Dates.DateTime, y::Dates.Year) in Dates at /usr/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:24
[42] +(x::Dates.DateTime, y::Dates.Period) in Dates at /usr/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:76
[43] +(B::BitArray{2}, J::LinearAlgebra.UniformScaling) in LinearAlgebra at /usr/share/julia/stdlib/v1.4/LinearAlgebra/src/uniformscaling.jl:115
[44] +(a::Pkg.Resolve.VersionWeight, b::Pkg.Resolve.VersionWeight) in Pkg.Resolve at /usr/share/julia/stdlib/v1.4/Pkg/src/Resolve/versionweights.jl:22
[45] +(a::Pkg.Resolve.FieldValue, b::Pkg.Resolve.FieldValue) in Pkg.Resolve at /usr/share/julia/stdlib/v1.4/Pkg/src/Resolve/fieldvalues.jl:43
[46] +(x::ChainRulesCore.NotImplemented, ::ChainRulesCore.NotImplemented) in ChainRulesCore at /home/mendez/.julia/packages/ChainRulesCore/8NXnp/src/t